## Batching

In [ ]:
def fany_ml_logic(sample):
    pass

print("\nBatch size 1")
# Instead of iterating over the entire dataset
# this is baych size 1
for epoch in [1,2,3]:
    for idx,row in df.iterrows():
        print(row.size)
        fany_ml_logic(row)
        break
print("\nBatch size Max")
# This would be ideal if you have infinite memory
# -> You dont
# Load all data at once into ml model
fany_ml_logic(df)
print(len(df),df.iloc[0].size)
print("\nProper Batching with size 8")
# You iterate over the datset in chunks
# good trade off between waiting for memory loading and processing
chunk_size=8
for epoch in [1,2,3]:
    for chunk_start in range(0,len(df),chunk_size):
        chunk=df.iloc[chunk_start:chunk_start+chunk_size]
        fany_ml_logic(chunk)
        print(len(chunk),chunk.iloc[0].size)
        break

In [31]:
# Custom Dataset Class
## Needs ATLEAST 3 class methods
## __init__, __len__, __getitem__
from torch.utils.data import Dataset
class CustomStarDataset(Dataset):
    # This loads the data and converts it, make data ready
    def __init__(self):
        # load data
        self.df=df
        # extract labels
        self.df_labels=targetsComps-1
        # drop non numeric columns to make tutorial simpler, in real life do categorical encoding
        #self.df=df.drop(columns=['Type','Color','Spectral_Class'])
        # conver to torch dtypes
        self.dataset=torch.tensor(self.df.to_numpy()).float()

        self.labels=torch.tensor(self.df_labels.to_numpy().reshape(-1)).long()
    
    # This returns the total amount of samples in your Dataset
    def __len__(self):
        return len(self.dataset)
    
    # This returns given an index the i-th sample and label
    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]

In [ ]:
# Create our dataset
ds=CustomStarDataset()
ds

In [ ]:
ds[0][0].shape # this gives the first value of the dataset and label at the end

#### Dataloader

In [35]:
from torch.utils.data import DataLoader
# create a dataloader
dl=DataLoader(ds,batch_size=32, num_workers=0, shuffle=False,drop_last=True)
#num workers controls parallelism

In [ ]:
# use dataloader
counter=0
for epoch in [1,2,3]:
    for batch,label in dl:
        if counter==0:
            print(batch,label)
            counter+=1
        print(batch.size(),label.size())

In [ ]:
for batch,label in dl:
    print(batch.size(),label.size(),batch[0], label)

In [ ]:
batch[1]

### Model

In [39]:
# Make simple Enum for code clarity
class DatasetType(Enum):
    TRAIN = 1
    TEST = 2
    VAL = 3

# Again create a Dataset but this time, do the split in train test val
class CustomStarDataset(Dataset):
    def __init__(self):
        # load data and shuffle, befor splitting
        self.df=df.sample(frac=1, random_state=27)
        train_split=0.8
        val_split=0.9
        self.df_labels=targetsComps-1
        # drop non numeric columns, in real life do categorical encoding
        #self.df=df
        # split pointf.df
        self.train, self.val, self.test = np.split(self.df, [int(train_split*len(self.df)), int(val_split*len(self.df))])
        self.train_labels, self.val_labels, self.test_labels = np.split(self.df_labels, [int(train_split*len(self.df)), int(val_split*len(self.df))])
        # do the feature scaling only on the train set!
        self.scaler=preprocessing.StandardScaler().fit(self.train)
        for data_split in [ self.train, self.val, self.test]:
           data_split=self.scaler.transform(data_split)
        # convet labels to 1 hot

            
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]
    
    def set_fold(self,set_type):
        # Make sure to call this befor using the dataset
        if set_type==DatasetType.TRAIN:
            self.dataset,self.labels=self.train,self.train_labels
        if set_type==DatasetType.TEST:
            self.dataset,self.labels=self.test,self.test_labels
        if set_type==DatasetType.VAL:
            self.dataset,self.labels=self.val,self.val_labels
        # Convert the datasets and the labels to pytorch format
        # Also use the StdScaler on the training set
        self.dataset=torch.tensor(self.scaler.transform(self.dataset)).float()
        self.labels=torch.tensor(self.labels.to_numpy().reshape(-1)).long()

        return self
        
dataset=CustomStarDataset()

In [40]:
# # Make simple Enum for code clarity
# class DatasetType(Enum):
#     TRAIN = 1
#     TEST = 2
#     #VAL = 3

# # Again create a Dataset but this time, do the split in train test val
# class CustomStarDataset(Dataset):
#     def __init__(self):
#         # load data and shuffle, befor splitting
#         self.df=df.sample(frac=1, random_state=27)
#         train_split=0.8
#         #val_split=0.9
#         self.df_labels=targetsComps-1
#         # drop non numeric columns, in real life do categorical encoding
#         #self.df=df
#         # split pointf.df
#         self.train, self.test = np.split(self.df, [int(train_split*len(self.df))])
#         self.train_labels, self.test_labels = np.split(self.df_labels, [int(train_split*len(self.df))])
#         # do the feature scaling only on the train set!
#         self.scaler=preprocessing.StandardScaler().fit(self.train)
#         for data_split in [ self.train, self.test]:
#            data_split=self.scaler.transform(data_split)
#         # convet labels to 1 hot

            
#     def __len__(self):
#         return len(self.dataset)

#     def __getitem__(self, idx):
#         return self.dataset[idx],self.labels[idx]
    
#     def set_fold(self,set_type):
#         # Make sure to call this befor using the dataset
#         if set_type==DatasetType.TRAIN:
#             self.dataset,self.labels=self.train,self.train_labels
#         if set_type==DatasetType.TEST:
#             self.dataset,self.labels=self.test,self.test_labels
#         # if set_type==DatasetType.VAL:
#         #     self.dataset,self.labels=self.val,self.val_labels
#         # Convert the datasets and the labels to pytorch format
#         # Also use the StdScaler on the training set
#         self.dataset=torch.tensor(self.scaler.transform(self.dataset)).float()
#         self.labels=torch.tensor(self.labels.to_numpy().reshape(-1)).long()
    
      

#         return self
        
# dataset=CustomStarDataset()

In [ ]:
%%time
# This is faster then recreating the dataset
# but you could also simply create it 3 times
# Or use PyTorchs build in train,test,val split
train=copy.deepcopy(dataset).set_fold(DatasetType.TRAIN)
test=copy.deepcopy(dataset).set_fold(DatasetType.TEST)
val=copy.deepcopy(dataset).set_fold(DatasetType.VAL)
for i in [train,test]:  #[train,test,val]:
    print(i.__getitem__(0)[1])


In [ ]:
len(train),len(test), len(val)

#### --------------------------------

In [43]:
# BATCH_SIZE=128

# class SimpleModel(nn.Module):
#     def __init__(self, train, test, val):
#         super(SimpleModel, self).__init__()
#         self.train_ds = train
#         self.val_ds = val
#         self.test_ds = test
#         # Define PyTorch model
#         classes = 80
#         features = 310
#         self.model = nn.Sequential(
#             #nn.Flatten(), # we probably don't need to flatten as the data is already flat
#             nn.Linear(features, 128),
#             nn.ReLU(),
#             #nn.Dropout(0.1),
#             #nn.Linear(128, 64),
#             #nn.ReLU(),
#             #nn.Dropout(0.1),
#             nn.Linear(128, classes),
#         )
#         self.accuracy = Accuracy(num_classes=80,task='multiclass')

#     def forward(self, x):
#         x = self.model(x)
#         return x #torch.nn.functional.log_softmax(x, dim=1)

#     def training_step(self, batch):
#         x, y = batch # inputs, targets
#         logits = self(x)
#         loss = nn.CrossEntropyLoss()(logits, y)
#         return loss

#     def validation_step(self, batch, print_str="val"):
#         x, y = batch #inputs, targets
#         logits = self(x)
#         loss = nn.CrossEntropyLoss()(logits, y)  # 
#         preds = torch.argmax(logits, dim=1)
#         self.accuracy(preds, y)
#         return loss

#     def test_step(self, batch):
#         return self.validation_step(batch, print_str='test')

#     def configure_optimizers(self):
#         return optim.Adam(self.parameters(), lr=0.001)

#     def train_dataloader(self):
#         return DataLoader(self.train_ds, batch_size=BATCH_SIZE, shuffle=False)

#     def val_dataloader(self):
#         return DataLoader(self.val_ds, batch_size=BATCH_SIZE, shuffle=False)

#     def test_dataloader(self):
#         return DataLoader(self.test_ds, batch_size=BATCH_SIZE, shuffle=False)

# # Define your data loaders for train, val, and test

# # Start the training loop
# model = SimpleModel(train, test, val)
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Define a training loop for a fixed number of epochs (e.g., 50)
# max_epochs = 3
# #Stuff to store
# train_losses=np.zeros(max_epochs)
# test_losses=np.zeros(max_epochs)

# for epoch in range(max_epochs):
#     train_loss=[]
#     model.train()
#     for batch in model.train_dataloader():
#         optimizer.zero_grad()
#         loss = model.training_step(batch) 
#         loss.backward()
#         optimizer.step()
#         train_loss.append(loss.item())
#     train_loss=np.mean(train_loss)

#     test_loss=[]
#     for batch in model.test_dataloader():
#         loss = model.training_step(batch)
#         test_loss.append(loss.item())
#     test_loss=np.mean(test_loss)
    
#     model.eval()
#     val_loss = 0
#     for batch in model.val_dataloader():
#         loss = model.validation_step(batch)
#         val_loss += loss.item()

#     #save losses
#     train_losses[epoch]=train_loss
#     test_losses[epoch]=test_loss
    
#     print(f"Epoch {epoch + 1}/{max_epochs}, Validation Loss: {val_loss:.4f}, train loss: {train_loss:.4f},test loss: {test_loss:.4f}")

# # Test on the test set
# test_loss = 0
# model.eval()
# for batch in model.test_dataloader():
#     loss = model.test_step(batch)
#     test_loss += loss.item()

# print(f"Test Loss: {test_loss:.4f}")


# 

# -----------------------------------

In [44]:
#build the model
model=nn.Sequential(
    nn.Linear(310,128),
    nn.ReLU(),
    nn.Linear(128,66)
)
#Loss and Optimizer
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)

In [45]:
from torch.utils.data import DataLoader, TensorDataset
nonscaledtrain=TensorDataset(torch.tensor(X_train).float(),torch.tensor(y_train.reshape(-1)).long())
nonscaledtrainloader = DataLoader(nonscaledtrain, batch_size=32, shuffle=True)


In [46]:
nonscaledtest=TensorDataset(torch.tensor(X_test).float(),torch.tensor(y_test.reshape(-1)).long())
nonscaledtestloader = DataLoader(nonscaledtest, batch_size=32, shuffle=True)

In [ ]:
torch.tensor(y_train.reshape(-1)).long()

In [ ]:
X_test

In [ ]:
torch.tensor(X_train).float().

In [ ]:
#THE NON SCALED DATA IS USED HERE

# Set early stopping parameters
patience = 50  # Number of epochs with no improvement after which training will be stopped
best_test_loss = float('inf')  # Initialize the best test loss
early_stopping_counter = 0  # Initialize the early stopping counter


#TRAIN THE MODEL
n_epochs=1000

#Stuff to store
train_losses=np.zeros(n_epochs)
test_losses=np.zeros(n_epochs)

for it in range(n_epochs):
  train_loss=[]
  for inputs,targets in nonscaledtrain:     
    #zero the parameter gradients
    optimizer.zero_grad()

    #forward pass
    outpus=model(inputs)
    loss=criterion(outpus,targets)

    #backward and optimize
    loss.backward()
    optimizer.step()

    train_loss.append(loss.item())

  #get train loss and test loss
  train_loss=np.mean(train_loss) #a little misleading

  test_loss=[]
  for inputs,targets in nonscaledtest:

    outputs=model(inputs)
    loss=criterion(outputs,targets)
    test_loss.append(loss.item())
  
  test_loss=np.mean(test_loss)

  #save losses
  train_losses[it]=train_loss
  test_losses[it]=test_loss

  print(f'Epoch {it+1}/{n_epochs},Train Loss: {train_loss:.4f},Test Loss:{test_loss:.4f}')

  # Check for early stopping
  if test_loss < best_test_loss:
      best_test_loss = test_loss
      early_stopping_counter = 0
  else:
      early_stopping_counter += 1
      if early_stopping_counter >= patience:
          print("Early stopping triggered. Training stopped.")
          break

In [ ]:
#Plot the train loss and test Loss per iteration
plt.plot(train_losses,label='train_losses')
plt.plot(test_losses,label='test loss')
plt.legend()
plt.show()

#### Loss data save

In [51]:
###### SAVING LOSS DATA

# losses_data = {
#     'Epoch': list(range(1, n_epochs + 1)),
#     'Train Loss': train_losses,
#     'Test Loss': test_losses
# }

# # Create a DataFrame from the dictionary
# losses_df = pd.DataFrame(losses_data)

# # Save the DataFrame to a CSV file
# losses_df.to_csv('ANN_comp_analysis_losses.csv', index=False)

# Train and Test ACC

In [ ]:
n_correct=0.
n_total=0.

for inputs, targets in nonscaledtrainloader:

  #forward pass
  outputs=model(inputs)

  #get prodictions 
  #torch.max returns both max and argmax
  _, predictions=torch.max(outputs,1)
  #predictions=torch.max(outputs)

  #update counts
  n_correct += (predictions==targets).sum().item()
  n_total += targets.shape[0]

train_acc=n_correct/n_total

n_correct=0.
n_total=0.

for inputs, targets in nonscaledtestloader:

  #forward pass
  outputs=model(inputs)

  #get prediction
  #torch.max returns both max and argmax
  _,predictions=torch.max(outputs,1)

  #update counts
  n_correct += (predictions==targets).sum().item()
  n_total += targets.shape[0]

test_acc=n_correct/n_total

print(f'Train acc {train_acc:.4f}, test acc {test_acc:.4f}')

In [ ]:
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc



def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module, 
               accuracy_fn):
  


  
  """Returns a dictionary containing the results of model predicting on data_loader."""
 
  loss, acc = 0, 0
  model.eval()
  with torch.inference_mode():
    for X, y in data_loader:
      # Make predictions
      y_pred = model(X)

      # Accumulate the loss and acc values per batch
      loss += loss_fn(y_pred, y)
      acc += accuracy_fn(y_true=y,
                         y_pred=y_pred.argmax(dim=1))

    # Scale loss and acc to find the average loss/acc per batch
    loss /= len(data_loader)
    acc /= len(data_loader)

    

  return {"model_name": model.__class__.__name__, # only works when model was created with a class
          "model_loss": loss.item(),
          "model_acc": acc}

# Calculate model 0 results on test dataset
model_0_results = eval_model(model=model,
                             data_loader=nonscaledtestloader,
                             loss_fn=criterion, 
                             accuracy_fn=accuracy_fn)
model_0_results